<a href="https://colab.research.google.com/github/victoriaporter58/Airline-Sentiment-Analysis-using-Machine-Learning/blob/main/SVM_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Base SVM Model
(Support Vector Machine)
Our Non-Neural model

##Import Drive
Comment out if necceseray

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##Import libraries

In [ ]:
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm
from scipy import sparse
from datetime import datetime
import pandas as pd
import numpy as np

##Read in cleaned tweet data

For our baseline model we are only going to consider the airline sentiment and the tweet text.

In [ ]:
#path for the Cleaned.csv file which in the current case is in a folder called "project folder" in the top level of my personal google drive
tweets_data_path = '/content/gdrive/My Drive/project_folder/Cleaned.csv'

tweets = pd.read_csv(tweets_data_path, header=0)

##Fit the count vectorizer

In [ ]:
count_vectorizer = CountVectorizer(ngram_range=(1,2))
vectorized_data = count_vectorizer.fit_transform(tweets.text)
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))

##Convert sentiment into integers

In [ ]:
def sentiment2target(sentiment):
    return {
        'negative': 0,
        'neutral': 1,
        'positive' : 2
    }[sentiment]
    
targets = tweets.airline_sentiment.apply(sentiment2target)

##Split data into train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(indexed_data, targets, test_size=0.2, random_state=1)
data_train_index = X_train[:,0]
X_train = X_train[:,1:]

data_test_index = X_test[:,0]
X_test = X_test[:,1:]

##Generate and train the model

In [ ]:

clf = OneVsRestClassifier(svm.SVC(gamma=1, C=0.1, probability=True, class_weight='balanced', kernel='linear'))
start_train_time = datetime.now()
clf_output = clf.fit(X_train, y_train)
end_train_time = datetime.now()
print('Training complete:',end_train_time-start_train_time)

Training complete: 0:04:05.955883


##Test the model and generate analysis metrics

In [ ]:
start_test_time = datetime.now()
predictions = clf.predict(X_test)
end_test_time = datetime.now()
expected = y_test.tolist()

print("Testing time:", end_test_time-start_test_time)
print("Accuracy: ", accuracy_score(expected, predictions), "\n")
print("Classification Report:")
print(classification_report(expected, predictions), "\n")
print("Confusion Matrix:")
print(confusion_matrix(expected, predictions))

Testing time: 0:00:09.737372
Accuracy:  0.7766393442622951 

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.83      0.86      1826
           1       0.56      0.68      0.62       611
           2       0.72      0.70      0.71       491

    accuracy                           0.78      2928
   macro avg       0.72      0.74      0.73      2928
weighted avg       0.79      0.78      0.78      2928
 

Confusion Matrix:
[[1512  245   69]
 [ 130  418   63]
 [  65   82  344]]


##Retrieve requirements

In [ ]:
#pip freeze > requirements.txt

In [ ]:
#cat requirements.txt